# Integrações com o RecipeNLG - Equipe GPALT (Analistas de Cardápios)

Ativando uma conexão de banco de dados em memória usando o SGBD H2:

In [1]:
%defaultDatasource jdbc:h2:mem:db


# Importando Tabelas dos dados originais do RecipeNLG

### NERs

Os NERs foram separados anteriormente. A visualização (bem como a contagem) dos que mais aparecem pode ser realizada a partir dos dados obtidos:

In [2]:
DROP TABLE IF EXISTS NERs;

CREATE TABLE NERs (
  NER VARCHAR(200),
  Quantidade INT,
  PRIMARY KEY (NER)
) AS SELECT
  NER, COUNT(*) Quantidade
FROM CSVREAD('../data/interim/recipenlg/ners.csv') GROUP BY NER;


Os NERs com mais de 10 mil usos nas receitas terão sua integração garantida. Uma menção importante é ao ingrediente de NER \u00bc (símbolo de 1/4), que aparece em inúmeras receitas. Nesse caso, o ingrediente criado "Unknown" foi útil, porque ele permite associar esse NER a algo desconhecido, evitando que as receitas sejam possivelmente descartadas (a quantidade de usos desse NER nas receitas é grande).

In [3]:
SELECT NER
FROM NERs
GROUP BY NER
HAVING Quantidade >= 10000
ORDER BY Quantidade DESC;


### Receitas inconsistentes

Com o processamento feito na base, agora a relação de ingredientes e receitas dentro da própria base do RecipeNLG está pronta. Como nos dados finais os ingredientes serão integrados com o FooDB, é preciso lidar com receitas que tenham ingredientes não integrados. Nesse caso, foi decidido que elas serão apagadas. Isso porque, prezando por uma maior consistência dos dados, não faz sentido considerar apenas partes das receitas.

Nessa etapa, houve dificuldade em importar a grande quantidade de dados. A solução encontrada foi fazer uma seleção junto à importação dos dados, filtrando somente o necessário. Encontrando as receitas que devem ser apagadas:

In [4]:
DROP TABLE IF EXISTS AssociacaoComFooDB;
-- Todas as receitas cujos ingredientes foram padronizados
DROP TABLE IF EXISTS ReceitasTotais;
-- Todas as receitas com um ou mais ingredientes não integráveis
DROP TABLE IF EXISTS ReceitasErradas;

CREATE TABLE AssociacaoComFooDB (
  Nome VARCHAR(100),
  FooDB VARCHAR(100),
  PRIMARY KEY (Nome)
) AS SELECT
  nome_ingrediente, nome_foodb
FROM CSVREAD('../data/external/associacaofinal.csv');

CREATE TABLE ReceitasTotais (
  ID INT,
  PRIMARY KEY (ID)
) AS SELECT DISTINCT
  IDReceita
FROM CSVREAD('../data/interim/recipenlg/ingredientesnasreceitas.csv');

CREATE TABLE ReceitasErradas (
  ID INT,
  PRIMARY KEY (ID)
) AS SELECT DISTINCT
  IDReceita
FROM CSVREAD('../data/interim/recipenlg/ingredientesnasreceitas.csv') WHERE Ingrediente NOT IN (SELECT Nome FROM AssociacaoComFooDB);


Visualizando:

In [5]:
SELECT * FROM ReceitasTotais LIMIT 10;


In [6]:
SELECT * FROM ReceitasErradas LIMIT 10;


Importando as informações das receitas:

In [7]:
DROP TABLE IF EXISTS NomesReceitas;

CREATE TABLE NomesReceitas (
  Id INT,
  Nome VARCHAR(350),
  BancoOriginal VARCHAR(50),
  Origem VARCHAR(50),
  PRIMARY KEY (Id)
) AS SELECT
  ID, Nome, BancoOriginal, Origem
FROM CSVREAD('../data/interim/recipenlg/recipes.csv');


Visualizando:

In [8]:
SELECT * FROM NomesReceitas LIMIT 10;


Agora, é preciso filtrar as receitas que estão certas. Isso não foi possível de ser realizado de imediato porque não é possível dizer, sem processar todas as entradas de cada receita, se ela deve ser considerada ou não, já que todos os ingredientes devem ter associação. Assim, a diferença entre as duas tabelas corresponde às receitas que efetivamente serão usadas. Considerando somente as receitas consistentes:

In [9]:
DROP VIEW IF EXISTS DadosReceitasCertas;
DROP VIEW IF EXISTS ReceitasCertas;

CREATE VIEW ReceitasCertas AS
    SELECT * FROM ReceitasTotais
    EXCEPT
    SELECT * FROM ReceitasErradas;

CREATE VIEW DadosReceitasCertas AS
    SELECT NR.Id, NR.Nome, NR.BancoOriginal, NR.Origem
    FROM NomesReceitas NR, ReceitasCertas RC
    WHERE NR.Id = RC.ID
    ORDER BY NR.ID;


Visualizando:

In [10]:
SELECT * FROM ReceitasCertas LIMIT 10;


In [11]:
SELECT * FROM DadosReceitasCertas LIMIT 10;


Exportando e apagando tabelas que não serão mais usadas nesse momento:

In [12]:
CALL CSVWRITE('../data/interim/recipenlg/receitasfinais.csv',
              'SELECT * FROM DadosReceitasCertas');

DROP VIEW IF EXISTS DadosReceitasCertas;
DROP TABLE IF EXISTS NomesReceitas;


152954

### Ingredientes e Receitas

Falta agora ajustar os dados dos ingredientes nas receitas. Considerando a mesma lógica da importação das receitas e que NomesReceitas, agora, só tem os nomes das receitas consistentes:

In [13]:
DROP TABLE IF EXISTS IngredientesNasReceitasRecipe;

CREATE TABLE IngredientesNasReceitasRecipe (
  IDReceita INT,
  Quantidade DECIMAL(20,3),
  Unidade VARCHAR(20),
  Ingrediente VARCHAR(80),
  PRIMARY KEY (IDReceita, Ingrediente)
) AS SELECT
  IDReceita, Quantidade, Unidade, Ingrediente
-- Foram tentadas diferentes construções mas nenhuma delas terminava.
-- A solução encontrada foi fazer um LEFT JOIN com as receitas erradas nos campos onde não há match
FROM CSVREAD('../data/interim/recipenlg/ingredientesnasreceitas.csv') I LEFT JOIN ReceitasErradas R ON R.ID = I.IDReceita WHERE R.ID IS NULL;


Visualizando:

In [14]:
SELECT * FROM IngredientesNasReceitasRecipe LIMIT 10;


Caso a quantidade de um ingrediente seja 0, não se tem informação sobre a quantidade/unidade. Ajustando isso nos dados:

In [15]:
UPDATE IngredientesNasReceitasRecipe
SET Unidade = 'desconhecida'
WHERE Quantidade = 0;


Esses dados já são integráveis. Resta apenas ajustar ao modelo proposto, alterando os nomes dos ingredientes para os nomes na base do FooDB

In [16]:
DROP VIEW IF EXISTS RelacaoFood;

CREATE VIEW RelacaoFood AS
    SELECT I.IDReceita, SUM(I.Quantidade) Quantidade, I.Unidade, A.FooDB
    FROM IngredientesNasReceitasRecipe I, AssociacaoComFooDB A
    WHERE I.Ingrediente = A.Nome
    GROUP BY I.IDReceita, I.Unidade, A.FooDB
    ORDER BY I.IDReceita;


Visualizando:

In [17]:
SELECT * FROM RelacaoFood LIMIT 10;


Exportando a associação nova (agora final, apenas com as receitas com ingredientes 100% integrados):

In [18]:
CALL CSVWRITE('../data/interim/recipenlg/quantidadesfinais.csv',
              'SELECT * FROM RelacaoFood');


979780